In [2]:
import typing

import pandas as pd

import config
import src
import requests
import tqdm
import json
import numpy as np
import logging

#import cltrier_lib as lib

import yaml
pd.set_option('display.max_colwidth', 100) 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [3]:
CFG = config.Config()

In [3]:
MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'

options_zero = """
seed: 42
temperature: 0
"""

options_zero = yaml.safe_load(options_zero)

In [4]:
#load data
reddit3000: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/reddit3000.parquet')
reddit3000.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score
0,t3_1bx6qw,,100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713...",None,non-political,0
1,t1_c9b2nyd,I've wanted to watch this for a long time. I was also turned off by the country western aspect.,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10,"[""The user has wanted to watch something for a long time"", ""The country western aspect of someth...","[0.4254729151725769, -0.01032189279794693, -0.6786631941795349, 0.32541653513908386, 0.298534691...",non-political,non-political,0
2,t1_c9b30i1,You strike me as the type who would appreciate it. I would give it a go. This is also my first A...,100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None,"[""Magnolia has a similar structure to Altman's films with no lead character"", ""Boogie Nights and...","[0.7148110270500183, -0.15317122638225555, -0.5416110157966614, -0.1810915321111679, -0.36302334...",non-political,non-political,0
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never se...",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10,"[""Altman is famous for his ensemble casts"", ""The user has never seen an Altman film""]","[0.7345885634422302, -0.177871972322464, -0.609611451625824, -0.0007055625319480896, -0.08616860...",non-political,non-political,0
4,t3_omv7p,"Alright guys, little background about myself. I'm a good looking, 23 year old male, have had mod...",100sets,"Male, 23 years old. Going for 100 sets!",none,announcement,True,NaN,Keatonus,"[""The writer has had modest success with women in the past"", ""The writer lacks killer instinct a...","[0.5479655265808105, -0.197927325963974, -0.4277328550815582, 0.18708674609661102, 0.06689110398...",None,non-political,0


In [6]:
#select the rows in the dataset that have the a showName with more than cutoff comments:   
cutoff = 40
selectedposts = reddit3000[reddit3000['thread_title'].isin(reddit3000['thread_title'].value_counts()[reddit3000['thread_title'].value_counts() >= cutoff].index)]

In [7]:
selectedposts.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
1175,t3_17feeu,"I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appre...",android,What ROM are you currently running and what do you like/dislike about it?,none,question,True,NaN,RipperDaVe,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1176,t1_c84zkny,ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is sta...,android,What ROM are you currently running and what do you like/dislike about it?,t3_17feeu,answer,None,1.0,None,"[""AOKP paranoid android and CM10 can be combined"", ""The combination of AOKP paranoid android and...","[0.6152559518814087, -0.22806480526924133, -0.7964597344398499, 0.3017873466014862, 0.0959199368..."
1177,t1_c84znti,Isn't this kind of like PAC (Paranoid/AOKP/CM)?,android,What ROM are you currently running and what do you like/dislike about it?,t1_c84zkny,question,None,2.0,OmegaVesko,"[""This is similar to PAC (Paranoid/AOKP/CM).""]","[0.4787346422672272, -0.1805042177438736, -0.42209142446517944, 0.4346879720687866, 0.3241922259..."
1178,t1_c850f55,"Omg. That's what it should be called, way cooler than paranoidKANGdroid.",android,What ROM are you currently running and what do you like/dislike about it?,t1_c84znti,appreciation,None,3.0,None,"[""ParanoidKANGdroid is not as cool as the suggested name.""]","[0.35291609168052673, -0.2874097526073456, -0.6045865416526794, 0.23958219587802887, 0.192347049..."
1179,t1_c850if1,http://forum.xda-developers.com/showthread.php?t=2015607,android,What ROM are you currently running and what do you like/dislike about it?,t1_c850f55,elaboration,None,4.0,OmegaVesko,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."


In [8]:
selectedposts['thread_title'].value_counts()

thread_title
What ROM are you currently running and what do you like/dislike about it?          41
Get rid of unwanted system apps (adb shell pm hide)                                41
If your country has a 30-35 hour work week, what do you do with your free time?    41
Name: count, dtype: int64

In [7]:
#try new Achim approach of first expanding the meaning of the post based on context and then classifying the addition of new information per post:


In [5]:
SYSTEM_expansion: str = \
    """
# Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread:\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details.\n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exactly repeating the target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:
    """

In [7]:
#find out which posts are political, and which are not:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (reddit3000["body"], MODELsmall, options=options_zero)
    )
    for label, path in CFG.prompt_classify_files.items() if label in ['political_post']
}


classifying political_post:   0%|          | 0/3000 [00:00<?, ?it/s]

classifying political_post: 100%|██████████| 3000/3000 [20:19<00:00,  2.46it/s]  


In [11]:
predictions

{'political_post': 0                None
 1       non-political
 2       non-political
 3       non-political
 4                None
             ...      
 2995    non-political
 2996    non-political
 2997    non-political
 2998    non-political
 2999    non-political
 Name: political_post, Length: 3000, dtype: object}

In [8]:
#join to the dataset:
for label, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to include the label
    preds_df.columns = [f'Llama31_{label}_8b']
    reddit3000 = reddit3000.join(preds_df)

political_post
non-political    2470
political         219
Name: count, dtype: int64
------------------------------------------


In [9]:
reddit3000.loc[:, 'Llama31_political_fill_8b'] = reddit3000.loc[:, 'Llama31_political_post_8b'].fillna('non-political')
reddit3000.loc[:, 'Llama31_political_fill_8b_score'] = reddit3000.loc[:, 'Llama31_political_post_8b'].map({'non-political': 0, 'political': 1}).fillna(0).astype(int)

In [11]:
#save data:
reddit3000.to_parquet(f'{CFG.report_dir}/reddit3000.parquet')

In [10]:
reddit3000.groupby(['thread_title'])['Llama31_political_fill_8b_score'].sum().sort_values(ascending=False)

thread_title
Is there any reason auto insurance and/or health insurance rates should be determined by sex? Should both, neither, or one of them have different rates for the sexes?    18
In all these videos, why does the person refuse to roll down their window?                                                                                                14
What modern nation has been in the most wars? What modern nation has been in the least? Is there a nation that have never been at war?                                     8
Why does it seem everyone forgot Washington Legalized Marijuana also???                                                                                                    7
If your country has a 30-35 hour work week, what do you do with your free time?                                                                                            7
                                                                                                                          

In [12]:
#find threads with most political posts:
#select the rows in the dataset that are in a thread with more political posts than cutoff:   
# Define the cutoff value
cutoff_value = 8

# Group by 'thread_title' and count the number of political posts
thread_counts = reddit3000.groupby('thread_title')['Llama31_political_fill_8b_score'].sum()

# Filter the threads based on the cutoff value
pol_threads = thread_counts[thread_counts > cutoff_value].index

# Select the posts in the filtered threads
pol_posts = reddit3000[reddit3000['thread_title'].isin(pol_threads)]

# Display the filtered posts
print(pol_posts.loc[pol_posts['Llama31_political_fill_8b_score'] == 1, 'body'].to_list())

['Is there a reason for this? Is it to prevent the cop from doing something like reach in? Or is it to make it plain that you do not consent to any searches?', 'I Googled this and found [this forum of cops](http://forums.officer.com/t142318/) talking about it.\n\nSounds like most cops are going to retaliate. The Texas cop even says that he can arrest you for any offense other than speeding or open carry. Another says he will delay you until he gets a K9 unit (which we know often falsely alerts). \n\nCheck out the responses. Pretty shocking.', 'this is disgusting to see these people who are supposed to help protect us discuss ways to hurt someone who is simply acting within their rights.', '"Refusing to roll you window down more than two inches in and of itself is suspicious. It suggests the odor of something is present inside your car that you do not want the officer to smell, such as drugs or alcohol. It also suggests you may be using the angle of the glass and any reflections off it 

In [26]:
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODELsmall,
                                'system': SYSTEM_expansion,
                                'prompt': f'"Thread":\n<{pol_posts["body"][:5].to_list()}>, "Target reply":<<{pol_posts["body"][5:6].to_list()}>>',
                                'options': options_zero
                                }).json()

{'id': '7a3c09d2-9b63-11ef-b791-b5b8664f6fd3',
 'timestamp': '2024-11-05T10:48:22.659925',
 'model': 'llama3.1:8b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': "\n# Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread:\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details.\n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exactly repeating the target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:\n    "},
  {'r

In [27]:
pol_posts.head(10)

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score
1101,t3_14h54j,Is there a reason for this? Is it to prevent the cop from doing something like reach in? Or is i...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",none,question,True,NaN,None,"[""The position of hands on the steering wheel may be to prevent the cop from reaching in"", ""The ...","[0.3588005006313324, -0.27677515149116516, -0.5966639518737793, 0.4516356289386749, 0.1325093507...",political,political,1
1102,t1_c7d1n5n,I Googled this and found [this forum of cops](http://forums.officer.com/t142318/) talking about ...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t3_14h54j,answer,None,1.0,None,"[""Most cops are going to retaliate"", ""A Texas cop claims he can arrest you for any offense other...","[0.4589661657810211, -0.28177395462989807, -0.6206199526786804, 0.6636458039283752, 0.0873084366...",political,political,1
1103,t1_c7d7ykx,this is disgusting to see these people who are supposed to help protect us discuss ways to hurt ...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,negativereaction,None,2.0,mtmadhatt,"[""People in positions of power discussed ways to harm someone exercising their rights.""]","[0.2069886475801468, -0.37488916516304016, -0.5948599576950073, 0.45740818977355957, 0.280526936...",political,political,1
1104,t1_c7ecdxv,"""Refusing to roll you window down more than two inches in and of itself is suspicious. It sugges...",amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,elaboration,None,3.0,Reinmaker,"[""Refusing to roll down your window more than two inches is suspicious"", ""Not rolling down your ...","[0.1393248736858368, -0.19022396206855774, -0.5485777854919434, 0.3504222333431244, -0.010414749...",political,political,1
1105,t1_c7dn1a4,And they somehow have the gall to whine about how the public doesn't trust them.,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d7ykx,agreement,None,3.0,NeonDisease,"[""Some individuals have the audacity to complain about public distrust.""]","[0.24344639480113983, -0.2551920711994171, -0.5752944350242615, 0.41955870389938354, 0.361750006...",political,political,1
1106,t1_c7d8a77,Wow. WTF? \n\nYou have GOT to put that on [r/Bad_Cop_No_Donut](r/Bad_Cop_No_Donut\n)\n\nI haven...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,negativereaction,None,2.0,rondeline,"[""A police officer suggested manufacturing infractions as retaliation for not rolling down the w...","[0.3623582422733307, -0.36987245082855225, -0.6905738711357117, 0.48078420758247375, 0.173638343...",non-political,non-political,0
1107,t1_c7dh0hi,If you want to link a subreddit you just need to type this:\n\n /r/Bad_Cop_No_Donut\n\nAnd it...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d8a77,elaboration,None,3.0,-Daniel,"[""Typing '/r/subredditname' creates a clickable link in comments""]","[0.27559125423431396, -0.09081175923347473, -0.6441599726676941, 0.18922629952430725, 0.29462119...",non-political,non-political,0
1108,t1_c8ntl9z,"""any attitude increases the chance of a citation."" isnt it important to evenly enforce the law? ...",amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,negativereaction,None,2.0,None,"[""Even enforcement of the law is important"", ""Not enforcing the law evenly shows a lack of care ...","[0.23230792582035065, -0.15393592417240143, -0.5654794573783875, 0.4555633068084717, 0.056398689...",political,political,1
1109,t1_c7d8g1b,>The Texas cop even says that he can arrest you for any offens

In [56]:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = selectedposts[:3].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            print(row['body'])
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            print('threadset=' + str(threadset))
            print('index=' + str(index))
            print(f'prompt: "Thread":\n<{df["body"][:threadset].to_list()}>, "Target reply":<<{df["body"][index]}>>')

100%|██████████| 1/1 [00:00<00:00, 106.67it/s]

I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appreciated, thanks!

Edit: Happy to hear ideas from other phones as well, thanks for the input
threadset=1175
index=1176
prompt: "Thread":
<["I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appreciated, thanks!\n\nEdit: Happy to hear ideas from other phones as well, thanks for the input", 'ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is stable! The customization is incredible. On jellybean too.\n\n**EDIT:** link for OP, on the I747 GS3 \n\nhttp://forum.xda-developers.com/showthread.php?t=1944523', "Isn't this kind of like PAC (Paranoid/AOKP/CM)?"]>, "Target reply":<<ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is stable! The customization is incredible. On jellybean too.

**EDIT:** link for OP, on the I747 GS3 

http://forum.xda-developers.com/showthread.php?t=1944

In [ ]:
#Expand the polposts to include the thread context:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = pol_posts.loc[pol_posts.Llama31_political_fill_8b == 'political',:].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[[index, row['body']]],
                    columns=['index', 'post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[[index,
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_expansion,
                                    'prompt': f'"Thread":\n<{df["body"][:threadset].to_list()}>, "Target reply":<<{df["body"][index]}>>',
                                    'options': options_zero
                                    }).json()['response']                       
                        ]],
                        columns=['index','post_expansion']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")
    

100%|██████████| 2/2 [00:57<00:00, 28.70s/it]


In [70]:
chunked_result

[   index  \
 0   1892   
 
                                                                                         post_expansion  
 0  I think the idea that being constantly productive = successful is very American. The concept is/...  ,
    index  \
 0   1893   
 
                                                                                         post_expansion  
 0  The concept of success being tied to constant productivity is deeply ingrained in American cultu...  ,
    index  \
 0   1897   
 
                                                                                         post_expansion  
 0  It's ironic that it's often those with the least free time who don't know what to do with it, wh...  ,
    index  \
 0   1900   
 
                                                                                         post_expansion  
 0  In many European countries, people have more free time due to a shorter standard working week (o...  ,
    index  \
 0   1901   
 
        

In [ ]:
expanded_posts = pd.concat(chunked_result, ignore_index=True)
print(expanded_posts)

In [72]:
expanded_posts.post_expansion.to_list()

["I think the idea that being constantly productive = successful is very American. The concept is/was one that was long held as a  [Core American Value](http://talkingsociology.blogspot.com/2008/05/core-american-values-handout.html?m=1). From a young age we're taught that we need to always work hard in order to live a good life; however, this leads a lot of people to feel the way you feel: if you're not busy then you're floundering. You need to find hobbies that you enjoy simply for the benefit of you enjoying them. There's nothing wrong with spending your time doing something you might consider frivolous. It's your life - enjoy the free time you have.\n\nEdit: Added blog post for anyone interested.",
 "The concept of success being tied to constant productivity is deeply ingrained in American culture, which can lead people to feel like they're floundering if they're not busy. This mindset is partly a result of capitalism's emphasis on high-paying jobs and wealth as measures of success.

In [32]:
pol_posts.index

Index([1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112,
       1113, 1114, 1115, 1116, 1117, 1118, 1119, 1888, 1889, 1890, 1891, 1892,
       1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904,
       1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916,
       1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928,
       1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940,
       1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952,
       1953, 1954, 1955, 1956, 1957, 1958],
      dtype='int64')

In [88]:
expanded_posts['index']

0     1892
1     1893
2     1897
3     1900
4     1901
5     1902
6     1912
7     1916
8     1922
9     1101
10    1102
11    1103
12    1104
13    1105
14    1108
15    1109
16    1110
17    1111
18    1113
19    1114
20    1117
21    1118
22    1930
23    1932
24    1933
25    1934
26    1936
27    1937
28    1938
29    1939
30    1940
31    1941
32    1942
33    1943
34    1949
35    1950
36    1953
37    1956
38    1958
Name: index, dtype: int64

In [15]:
expanded_posts.set_index(expanded_posts['index'], inplace=True)
pol_posts = pol_posts.join(expanded_posts['post_expansion'])

In [103]:
pol_posts.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score,post_expansion
1101,t3_14h54j,Is there a reason for this? Is it to prevent the cop from doing something like reach in? Or is i...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",none,question,True,NaN,None,"[""The position of hands on the steering wheel may be to prevent the cop from reaching in"", ""The ...","[0.3588005006313324, -0.27677515149116516, -0.5966639518737793, 0.4516356289386749, 0.1325093507...",political,political,1,Is there a reason for this? Is it to prevent the cop from doing something like reach in? Or is i...
1102,t1_c7d1n5n,I Googled this and found [this forum of cops](http://forums.officer.com/t142318/) talking about ...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t3_14h54j,answer,None,1.0,None,"[""Most cops are going to retaliate"", ""A Texas cop claims he can arrest you for any offense other...","[0.4589661657810211, -0.28177395462989807, -0.6206199526786804, 0.6636458039283752, 0.0873084366...",political,political,1,I Googled this and found [this forum of cops](http://forums.officer.com/t142318/) talking about ...
1103,t1_c7d7ykx,this is disgusting to see these people who are supposed to help protect us discuss ways to hurt ...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,negativereaction,None,2.0,mtmadhatt,"[""People in positions of power discussed ways to harm someone exercising their rights.""]","[0.2069886475801468, -0.37488916516304016, -0.5948599576950073, 0.45740818977355957, 0.280526936...",political,political,1,"""Cops discussing ways to retaliate against citizens for exercising their rights, such as refusin..."
1104,t1_c7ecdxv,"""Refusing to roll you window down more than two inches in and of itself is suspicious. It sugges...",amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,elaboration,None,3.0,Reinmaker,"[""Refusing to roll down your window more than two inches is suspicious"", ""Not rolling down your ...","[0.1393248736858368, -0.19022396206855774, -0.5485777854919434, 0.3504222333431244, -0.010414749...",political,political,1,"""Refusing to roll down your window more than two inches can be seen as suspicious by police beca..."
1105,t1_c7dn1a4,And they somehow have the gall to whine about how the public doesn't trust them.,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d7ykx,agreement,None,3.0,NeonDisease,"[""Some individuals have the audacity to complain about public distrust.""]","[0.24344639480113983, -0.2551920711994171, -0.5752944350242615, 0.41955870389938354, 0.361750006...",political,political,1,"They're discussing ways to hurt someone who is simply acting within their rights, and then have ..."


In [6]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assistant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""

In [105]:
pol_posts['post_expansion'].iloc[2]

'"Cops discussing ways to retaliate against citizens for exercising their rights, such as refusing to roll down their window more than two inches during a traffic stop, is disgusting and goes against the principles of protecting and serving the public."'

In [106]:
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODELsmall,
                                'system': SYSTEM_claim,
                                'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                        + f'Check whether your answer only consists of a list of claims. \n"Posts":\n<{pol_posts['post_expansion'].iloc[2]}>',
                                'options': options_zero
                                }).json()

{'id': '9b0af546-9b69-11ef-b791-b5b8664f6fd3',
 'timestamp': '2024-11-05T11:32:14.683455',
 'model': 'llama3.1:8b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': '\n        Instruction:\n\n        You are a text annotation assistant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.\n        Claims are defined as the main assertion or conclusion of an argument.\n        You summarize each claim into a short simple sentence.\n\n        Response format:\n\n        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].\n\n        Response format template:\n        \n        ["claim 1", "claim 2", ... "claim x"]\n\t'},
  {'role': 'user',
   'content': 'The following set of social media posts are replies 

In [17]:
#test zero temperature claims per comment on the expanded posts with political content:
chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(pol_posts.loc[pol_posts.Llama31_political_fill_8b == 'political',:].iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[[index,
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODELsmall,
                            'system': SYSTEM_claim,
                            'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                    + f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["post_expansion"]}>',
                            'options': options_zero
                            }).json()['response']                       
                ]],
                columns=['index', 'claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

32it [00:23,  1.38it/s]


In [ ]:
claims = pd.concat(chunked_result, ignore_index=True)
print(claims)

In [19]:
claims.set_index(claims['index'], inplace=True)
pol_posts = pol_posts.join(claims['claims'])


In [20]:
pol_posts.head(10)

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score,post_expansion,claims
1101,t3_14h54j,Is there a reason for this? Is it to prevent the cop from doing something like reach in? Or is i...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",none,question,True,NaN,None,"[""The position of hands on the steering wheel may be to prevent the cop from reaching in"", ""The ...","[0.3588005006313324, -0.27677515149116516, -0.5966639518737793, 0.4516356289386749, 0.1325093507...",political,political,1,Is there a reason for this? Is it to prevent the cop from doing something like reach in? Or is i...,"[""The police car has a partition to prevent the officer from reaching in"", ""The partition is to ..."
1102,t1_c7d1n5n,I Googled this and found [this forum of cops](http://forums.officer.com/t142318/) talking about ...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t3_14h54j,answer,None,1.0,None,"[""Most cops are going to retaliate"", ""A Texas cop claims he can arrest you for any offense other...","[0.4589661657810211, -0.28177395462989807, -0.6206199526786804, 0.6636458039283752, 0.0873084366...",political,political,1,I Googled this and found [this forum of cops](http://forums.officer.com/t142318/) talking about ...,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops..."
1103,t1_c7d7ykx,this is disgusting to see these people who are supposed to help protect us discuss ways to hurt ...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,negativereaction,None,2.0,mtmadhatt,"[""People in positions of power discussed ways to harm someone exercising their rights.""]","[0.2069886475801468, -0.37488916516304016, -0.5948599576950073, 0.45740818977355957, 0.280526936...",political,political,1,"Cops discussing ways to retaliate against citizens who exercise their rights, such as refusing t...","[""Cops are discussing ways to retaliate against citizens"", ""exercising their rights is disgustin..."
1104,t1_c7ecdxv,"""Refusing to roll you window down more than two inches in and of itself is suspicious. It sugges...",amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,elaboration,None,3.0,Reinmaker,"[""Refusing to roll down your window more than two inches is suspicious"", ""Not rolling down your ...","[0.1393248736858368, -0.19022396206855774, -0.5485777854919434, 0.3504222333431244, -0.010414749...",political,political,1,"""Refusing to roll down your window more than two inches can be seen as suspicious behavior by an...","[""Refusing to roll down your window more than two inches can be seen as suspicious behavior by a..."
1105,t1_c7dn1a4,And they somehow have the gall to whine about how the public doesn't trust them.,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d7ykx,agreement,None,3.0,NeonDisease,"[""Some individuals have the audacity to complain about public distrust.""]","[0.24344639480113983, -0.2551920711994171, -0.5752944350242615, 0.41955870389938354, 0.361750006...",political,political,1,"They're discussing ways to hurt someone who is simply acting within their rights, and then have ...","[""People are being targeted for exercising their rights"", ""The public lacks trust in certain ind..."
1106,t1_c7d8a77,Wow. WTF? \n\nYou have GOT to put that on [r/Bad_Cop_No_Donut](r/Bad_Cop_No_Donut\n)\n\nI haven...,amifreetogo,"In all these videos, why does the person refuse to roll down their window?",t1_c7d1n5n,negativereaction,None,2.0,rondeline,"[""A police officer suggested manufacturing infractions as retaliation for not rolling down the w...","[0.3623582422733307, -0.36987245082855225, -0.6905738711357117, 0.48078420758247375,

In [21]:
#save data:
pol_posts.to_parquet(f'{CFG.report_dir}/reddit3000_pol_posts.parquet')

In [5]:
#ok, to determine the diversity of these claims, first group the claims per topic:


In [8]:
claims['claims'] = claims['claims'].str.split('",')
claims = claims.explode('claims')
claims = claims.reset_index(drop=True)
claims = claims.drop_duplicates()
print(claims)

    index  \
0    1101   
1    1101   
2    1102   
3    1102   
4    1102   
..    ...   
84   1956   
85   1956   
86   1958   
87   1958   
88   1958   

                                                                                              claims  \
0                           ["The police car has a partition to prevent the officer from reaching in   
1                                       "The partition is to indicate lack of consent for searches"]   
2   ["Cops are discussing retaliating against drivers who exercise their rights during traffic stops   
3                "A Texas cop claims he can arrest for any offense other than speeding or open carry   
4                                      "Some cops suggest delaying drivers until a K9 unit arrives"]   
..                                                                                               ...   
84               ["Non-malicious discrimination can occur through actuarial tables and market forces   
85         

In [7]:
SYSTEM_topic: str = \
    """
        # Instruction\n\nYour task is to identify the main discussion topic for a claim. The topic should reflect the core subject or theme that the claim relates to. Follow these steps:\n\n1. You will be provided with a claim enclosed in chevrons <...> .\n2. You determine the primary topic in a concise manner (e.g., 'climate change', 'economic policy', 'sports').\n3. Ensure the topic is general enough to encompass the main idea but specific enough to be meaningful.\n\nIf no topic can be identified, return an empty list []. Your respond consists solely of the topic.
	"""

In [8]:
SYSTEM_thread: str = \
    """
# Instruction

Identify the main discussion topic for each claim in a thread. The topic should capture the core subject or theme of the claim. Follow these steps:

1. You will receive a JSON object containing a thread of claims along with their respective indices enclosed in chevrons `<...>`.
2. For each claim, identify the primary topic using concise labels (e.g., 'climate change', 'economic policy', 'sports'). If multiple claims address similar topics, determine the central topic and apply this same topic to all related claims. Ensure that each claim is assigned only one topic.
3. Ensure each topic is broad enough to capture the main idea but specific enough to be meaningful.
4. If a topic cannot be identified, return an empty value `[]`.

Response format in JSON:

[
    {
        "claim_index": "1",
        "topic": "Topic 1"
    },
    {
        "claim_index": "2",
        "topic": "Topic 2"
    }
]
	"""

In [53]:
SYSTEM_thread2: str = \
    """
# Instruction

Identify the topic-clusters in a set of claims. The topics should capture the core subject or theme of the claims. Follow these steps:

1. You will receive a JSON object containing a thread of claims along with their respective indices enclosed in chevrons `<...>`.
2. Find the dominant topic-clusters for the claims. Ensure that each claim is assigned to only one topic.
3. Give a numbered value to each of the identified topics.
4. If a topic cannot be identified, return an empty value `[]`.

Response format in JSON:

[
    {
        "claim_index": "1",
        "topic": "Topic 1"
    },
    {
        "claim_index": "2",
        "topic": "Topic 2"
    }
]
	"""

In [ ]:
SYSTEM_thread3: str = \
    """
# Instruction

Identify the topic-clusters in a set of claims. The topics should capture the core subject or theme of the claims. Follow these steps:

1. You will receive a JSON object containing a thread of claims along with their respective indices enclosed in chevrons `<...>`.
2. Find the dominant topic-clusters for the claims. Ensure that each claim is assigned to only one topic.
3. Give a numbered value to each of the identified topics.
4. If a topic cannot be identified, return an empty value `[]`.
5. Your response consists of a JSON object with the claim index and the corresponding topic-cluster number.

Response format in JSON:

[
    {
        "claim_index": "1",
        "topic-cluster": "1"
    },
    {
        "claim_index": "2",
        "topic-cluster": "1"
    },
    {
        "claim_index": "3",
        "topic-cluster": "2"
    }
]
	"""

In [60]:
SYSTEM_thread_level: str = \
    """
# Instruction

Identify the topic-clusters and sub-clusters in a set of claims. The topics should capture the broad subject or theme of the claims. Sub-topics can be specific aspects, elaborations, additions or context to a topic. Follow these steps:

1. You will receive a JSON object containing a thread of claims along with their respective indices enclosed in chevrons `<...>`.
2. Find the dominant topic-clusters and sub-clusters for the claims. Ensure that each claim is assigned to only one topic and sub-topic.
3. If a topic or sub-topic cannot be identified, return an empty value `[]`.
4. Your response consists of a JSON object with the claim index and the corresponding topic-cluster and sub-cluster numbers.

Response format in JSON:

[
    {
        "claim_index": "1",
        "topic": "Topic 1",
        "sub-topic": "Sub-topic 1"	
    },
    {
        "claim_index": "2",
        "topic": "Topic 2",
        "sub-topic": "Sub-topic 2"
    }
]
	"""

In [25]:
#test topic mining
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODELsmall,
                                'system': SYSTEM_topic,
                                'options': options_zero,
                                'prompt': f'"Claim":\n<{claims.loc[:,'claims'][1]}>'
                                }).json()['response']

"'police procedure'"

In [36]:
chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(claims.iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[[row['index'],
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODELsmall,
                            'system': SYSTEM_topic,
                            'prompt': f'"Claim":\n<{row['claims']}>',
                            'options': options_zero
                            }).json()['response']                       
                ]],
                columns=['index', 'claim_topic']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

89it [00:27,  3.20it/s]


In [27]:
topics = pd.concat(chunked_result, ignore_index=True)
print(topics)

    index                        claim_topic
0    1101                  "police vehicles"
1    1101                 'police procedure'
2    1102  "Cop behavior and accountability"
3    1102                    "Police powers"
4    1102          "Law Enforcement Tactics"
..    ...                                ...
84   1956                   "Discrimination"
85   1956                  "Economic policy"
86   1958                   'discrimination'
87   1958                   'discrimination'
88   1958                  "Economic policy"

[89 rows x 2 columns]


In [28]:
claims = claims.join(topics['claim_topic'])

In [29]:
claims

,index,claims,claim_topic
0,1101,"[""The police car has a partition to prevent the officer from reaching in","""police vehicles"""
1,1101,"""The partition is to indicate lack of consent for searches""]",'police procedure'
2,1102,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","""Cop behavior and accountability"""
3,1102,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""Police powers"""
4,1102,"""Some cops suggest delaying drivers until a K9 unit arrives""]","""Law Enforcement Tactics"""
...,...,...,...
84,1956,"[""Non-malicious discrimination can occur through actuarial tables and market forces","""Discrimination"""
85,1956,"""Economic views are more relevant than gender equality views""]","""Economic policy"""
86,1958,"[""The target reply is referring to non-malicious discrimination",'discrimination'
87,1958,"""Non-malicious discrimination can occur through actuarial tables or market forces",'discrimination'


In [30]:
claims.claim_topic.value_counts()

claim_topic
[]                        9
"Economic policy"         3
'healthcare policy'       3
healthcare policy         3
"Evidence reliability"    2
                         ..
'Insurance pricing'       1
'healthcare'              1
"Driving Behavior"        1
'driver behavior'         1
"Discrimination"          1
Name: count, Length: 71, dtype: int64

In [70]:
claims['index'].value_counts()

index
1930    8
1943    7
1932    5
1117    5
1933    5
1950    5
1113    4
1103    4
1937    4
1958    3
1942    3
1941    3
1949    3
1102    3
1104    3
1108    3
1956    2
1105    2
1109    2
1112    2
1101    2
1114    1
1111    1
1110    1
1118    1
1939    1
1940    1
1936    1
1934    1
1935    1
1938    1
1953    1
Name: count, dtype: int64

In [31]:
claims = claims.join(pol_posts['thread_title'], how='left', on='index')

In [32]:
#save data:
claims.to_parquet(f'{CFG.report_dir}/reddit3000_pol_posts_claims.parquet')

In [38]:
#load data:
pol_posts: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/reddit3000_pol_posts.parquet')
claims: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/reddit3000_pol_posts_claims.parquet')

In [72]:
claims.thread_title.value_counts()

thread_title
Is there any reason auto insurance and/or health insurance rates should be determined by sex? Should both, neither, or one of them have different rates for the sexes?    55
In all these videos, why does the person refuse to roll down their window?                                                                                                34
Name: count, dtype: int64

In [39]:
options_large = """
seed: 42
temperature: 0
num_predict: 2000
"""

options_large = yaml.safe_load(options_large)

In [12]:
claims.head()

,index,claims,claim_topic,thread_title
0,1101,"[""The police car has a partition to prevent the officer from reaching in","""police vehicles""","In all these videos, why does the person refuse to roll down their window?"
1,1101,"""The partition is to indicate lack of consent for searches""]",'police procedure',"In all these videos, why does the person refuse to roll down their window?"
2,1102,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","""Cop behavior and accountability""","In all these videos, why does the person refuse to roll down their window?"
3,1102,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""Police powers""","In all these videos, why does the person refuse to roll down their window?"
4,1102,"""Some cops suggest delaying drivers until a K9 unit arrives""]","""Law Enforcement Tactics""","In all these videos, why does the person refuse to roll down their window?"


In [13]:
claims.loc[:, 'claim_index'] = claims.index

In [14]:
claims.claim_index.value_counts()

claim_index
0     1
1     1
2     1
3     1
4     1
     ..
84    1
85    1
86    1
87    1
88    1
Name: count, Length: 89, dtype: int64

In [16]:
#test topic mining
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODELsmall,
                                'system': SYSTEM_thread2,
                                'options': options_zero,
                                'prompt': f'Only respond with the JSON without any comments or introduction,"Claim":\n<{claims.loc[:,['claim_index', 'claims']].iloc[:2].to_json(orient='records')}>'
                                }).json()['response']

'[\n    {\n        "claim_index": "<0>",\n        "topic": "Police Car Partition"\n    },\n    {\n        "claim_index": "<1>",\n        "topic": "Lack of Consent"\n    }\n]'

In [17]:
# Initialize an empty list to store the results
chunked_result: typing.List[dict] = []

# Define the GROUPER
GROUPER = 'thread_title'

# Group the data by 'thread_title'
groupeddata = claims.groupby(GROUPER)

# Process each group
for group, df in tqdm.tqdm(groupeddata):
    try:
        response = requests.post(
            'https://inf.cl.uni-trier.de/',
            json={
                'model': MODELsmall,
                'system': SYSTEM_thread2,
                'prompt': f'Only respond with the JSON without any comments or introduction, "Claims":\n<{df.loc[:,['claim_index', 'claims']].to_json(orient='records')}>',
                'options': options_large
                }
            ).json()
        

    except json.JSONDecodeError:
        print("Invalid JSON response, skipping to next batch")

100%|██████████| 2/2 [00:35<00:00, 17.94s/it]


In [19]:
response['response']

'[\n    {\n        "claim_index": "34",\n        "topic": "Health/Insurance and Ethnicity"\n    },\n    {\n        "claim_index": "35",\n        "topic": "Health/Insurance and Ethnicity"\n    },\n    {\n        "claim_index": "36",\n        "topic": "Health/Insurance and Ethnicity"\n    },\n    {\n        "claim_index": "37",\n        "topic": "Health/Insurance and Ethnicity"\n    },\n    {\n        "claim_index": "38",\n        "topic": "Auto Insurance and Ethnicity"\n    },\n    {\n        "claim_index": "39",\n        "topic": "Auto Insurance and Ethnicity"\n    },\n    {\n        "claim_index": "40",\n        "topic": "Health/Insurance and Ethnicity"\n    },\n    {\n        "claim_index": "41",\n        "topic": "Health/Insurance and Sex"\n    },\n    {\n        "claim_index": "42",\n        "topic": "Auto Insurance and Sex"\n    },\n    {\n        "claim_index": "43",\n        "topic": "Auto Insurance and Sex"\n    },\n    {\n        "claim_index": "44",\n        "topic": "Health/

In [20]:
try:
    json_obj = json.loads(response['response'])
    print(json.dumps(json_obj, indent=4))
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

[
    {
        "claim_index": "34",
        "topic": "Health/Insurance and Ethnicity"
    },
    {
        "claim_index": "35",
        "topic": "Health/Insurance and Ethnicity"
    },
    {
        "claim_index": "36",
        "topic": "Health/Insurance and Ethnicity"
    },
    {
        "claim_index": "37",
        "topic": "Health/Insurance and Ethnicity"
    },
    {
        "claim_index": "38",
        "topic": "Auto Insurance and Ethnicity"
    },
    {
        "claim_index": "39",
        "topic": "Auto Insurance and Ethnicity"
    },
    {
        "claim_index": "40",
        "topic": "Health/Insurance and Ethnicity"
    },
    {
        "claim_index": "41",
        "topic": "Health/Insurance and Sex"
    },
    {
        "claim_index": "42",
        "topic": "Auto Insurance and Sex"
    },
    {
        "claim_index": "43",
        "topic": "Auto Insurance and Sex"
    },
    {
        "claim_index": "44",
        "topic": "Health/Insurance and Sex"
    },
    {
        "cla

In [ ]:
#too many claims: output is too long:

In [21]:
json.loads(response['response'])



[{'claim_index': '34', 'topic': 'Health/Insurance and Ethnicity'},
 {'claim_index': '35', 'topic': 'Health/Insurance and Ethnicity'},
 {'claim_index': '36', 'topic': 'Health/Insurance and Ethnicity'},
 {'claim_index': '37', 'topic': 'Health/Insurance and Ethnicity'},
 {'claim_index': '38', 'topic': 'Auto Insurance and Ethnicity'},
 {'claim_index': '39', 'topic': 'Auto Insurance and Ethnicity'},
 {'claim_index': '40', 'topic': 'Health/Insurance and Ethnicity'},
 {'claim_index': '41', 'topic': 'Health/Insurance and Sex'},
 {'claim_index': '42', 'topic': 'Auto Insurance and Sex'},
 {'claim_index': '43', 'topic': 'Auto Insurance and Sex'},
 {'claim_index': '44', 'topic': 'Health/Insurance and Sex'},
 {'claim_index': '45', 'topic': 'Health/Insurance and Age'},
 {'claim_index': '46', 'topic': 'Healthcare System'},
 {'claim_index': '47', 'topic': "Women's Health Services"},
 {'claim_index': '48', 'topic': 'Auto Accidents and Socialization'},
 {'claim_index': '49', 'topic': 'Reckless Driving a

In [22]:
# Initialize an empty list to store the results
chunked_result: typing.List[dict] = []

for claim in json.loads(response['response']):
    chunked_result.append({
        'thread_title': group,
        'post_topic': claim.get('topic', 'unknown'),  # Use 'unknown' if 'topic' key is missing
        'claim': claim.get('claim', 'unknown')  # Use 'unknown' if 'claim' key is missing
    })


In [23]:
chunked_result

[{'thread_title': 'Is there any reason auto insurance and/or health insurance rates should be determined by sex? Should both, neither, or one of them have different rates for the sexes?',
  'post_topic': 'Health/Insurance and Ethnicity',
  'claim': 'unknown'},
 {'thread_title': 'Is there any reason auto insurance and/or health insurance rates should be determined by sex? Should both, neither, or one of them have different rates for the sexes?',
  'post_topic': 'Health/Insurance and Ethnicity',
  'claim': 'unknown'},
 {'thread_title': 'Is there any reason auto insurance and/or health insurance rates should be determined by sex? Should both, neither, or one of them have different rates for the sexes?',
  'post_topic': 'Health/Insurance and Ethnicity',
  'claim': 'unknown'},
 {'thread_title': 'Is there any reason auto insurance and/or health insurance rates should be determined by sex? Should both, neither, or one of them have different rates for the sexes?',
  'post_topic': 'Health/Insur

In [45]:
# Initialize an empty list to store the results
chunked_result: typing.List[dict] = []

# Define the GROUPER
GROUPER = 'thread_title'

# Group the data by 'thread_title'
groupeddata = claims.groupby(GROUPER)

# Process each group
for group, df in tqdm.tqdm(groupeddata):
    try:
        response = requests.post(
            'https://inf.cl.uni-trier.de/',
            json={
                'model': MODELsmall,
                'system': SYSTEM_thread2,
                'prompt': f'Only respond with the JSON without any comments or introduction, "Claims":\n<{df.loc[:,['claim_index', 'claims']].to_json(orient='records')}>',
                'options': options_large
                }
            ).json()
        
        # Check if 'response' key exists in the JSON response
        if 'response' in response:
            for claim in json.loads(response['response']):
                chunked_result.append({
                    'thread_title': group,
                    'post_topic': claim.get('topic', 'unknown'),  # Use 'unknown' if 'topic' key is missing
                    'claim_index': claim.get('claim_index', 'unknown')  # Use 'unknown' if 'claim' key is missing
                })
        else:
            print(f"Key 'response' not found in the response: {response}")
    
    except json.JSONDecodeError:
        print("Invalid JSON response, skipping to next batch")
    except requests.RequestException as e:
        print(f"Request failed: {e}")

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(result_df)


100%|██████████| 2/2 [00:41<00:00, 20.83s/it]

                                                                                           thread_title  \
0                            In all these videos, why does the person refuse to roll down their window?   
1                            In all these videos, why does the person refuse to roll down their window?   
2                            In all these videos, why does the person refuse to roll down their window?   
3                            In all these videos, why does the person refuse to roll down their window?   
4                            In all these videos, why does the person refuse to roll down their window?   
..                                                                                                  ...   
84  Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...   
85  Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...   
86  Is there any reason auto insuranc

In [47]:
#join to the dataset:
claims['claims'] = claims['claims'].str.replace(r'^\["', '', regex=True).str.replace(r'"\]$', '', regex=True).str.replace(r'"', '', regex=True).str.strip()
result_df['claim_index'] = result_df['claim_index'].astype(int)
result = pd.merge(claims, result_df, how='left', on='claim_index')


In [48]:
result

,index,claims,claim_topic,thread_title_x,claim_index,thread_title_y,post_topic
0,1101,The police car has a partition to prevent the officer from reaching in,"""police vehicles""","In all these videos, why does the person refuse to roll down their window?",0,"In all these videos, why does the person refuse to roll down their window?",Police Car Partition
1,1101,The partition is to indicate lack of consent for searches,'police procedure',"In all these videos, why does the person refuse to roll down their window?",1,"In all these videos, why does the person refuse to roll down their window?",Lack of Consent for Searches
2,1102,Cops are discussing retaliating against drivers who exercise their rights during traffic stops,"""Cop behavior and accountability""","In all these videos, why does the person refuse to roll down their window?",2,"In all these videos, why does the person refuse to roll down their window?",Retaliation Against Drivers
3,1102,A Texas cop claims he can arrest for any offense other than speeding or open carry,"""Police powers""","In all these videos, why does the person refuse to roll down their window?",3,"In all these videos, why does the person refuse to roll down their window?",Arrest Powers of Texas Cop
4,1102,Some cops suggest delaying drivers until a K9 unit arrives,"""Law Enforcement Tactics""","In all these videos, why does the person refuse to roll down their window?",4,"In all these videos, why does the person refuse to roll down their window?",K9 Unit Deployment
...,...,...,...,...,...,...,...
84,1956,Non-malicious discrimination can occur through actuarial tables and market forces,"""Discrimination""",Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,84,Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,Non-Malicious Discrimination through Actuarial Tables and Market Forces
85,1956,Economic views are more relevant than gender equality views,"""Economic policy""",Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,85,Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,Economic Views vs. Gender Equality Views
86,1958,The target reply is referring to non-malicious discrimination,'discrimination',Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,86,Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,Target Reply Referring to Non-Malicious Discrimination
87,1958,Non-malicious discrimination can occur through actuarial tables or market forces,'discrimination',Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,87,Is there any reason auto insurance and/or health insurance rates should be determined by sex? Sh...,Non-Malicious Discrimination through Actuarial Tables or Market Forces


In [49]:
len(result.claim_topic.unique())

71

In [50]:
result.post_topic.isna().sum()

np.int64(0)

In [ ]:
non_empty_rows = result[result['post_topic'].notna() & (result['post_topic'] != '')]


In [51]:
len(result.post_topic.unique())

70

In [ ]:
#changing the prompt somewhat reduced the number of topics but not enough to be useful

In [82]:
#try mining both topics and subtopics:

# Initialize an empty list to store the results
chunked_result: typing.List[dict] = []

# Define the GROUPER
GROUPER = 'thread_title'

# Group the data by 'thread_title'
groupeddata = claims.groupby(GROUPER)

# Process each group
for group, df in tqdm.tqdm(groupeddata):
    try:
        response = requests.post(
            'https://inf.cl.uni-trier.de/',
            json={
                'model': MODELsmall,
                'system': SYSTEM_thread_level,
                'prompt': f'Only respond with the JSON without any comments or introduction, "Claims":\n<{df.loc[:,['claim_index', 'claims']].to_json(orient='records')}>',
                'options': options_large
                }
            ).json()
        
        # Check if 'response' key exists in the JSON response
        if 'response' in response:
            for claim in json.loads(response['response']):
                chunked_result.append({
                    'thread_title': group,
                    'claim_topic_t': claim.get('topic', 'unknown'),  # Use 'unknown' if 'topic' key is missing
                    'claim_sub_topic_t': claim.get('sub-topic', 'unknown'),  # Use 'unknown' if 'topic' key is missing
                    'claim_index': claim.get('claim_index', 'unknown')  # Use 'unknown' if 'claim' key is missing
                })
        else:
            print(f"Key 'response' not found in the response: {response}")
    
    except json.JSONDecodeError:
        print("Invalid JSON response, skipping to next batch")
    except requests.RequestException as e:
        print(f"Request failed: {e}")

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(result_df.head(2))

100%|██████████| 2/2 [00:51<00:00, 25.82s/it]

                                                                 thread_title  \
0  In all these videos, why does the person refuse to roll down their window?   
1  In all these videos, why does the person refuse to roll down their window?   

          claim_topic_t claim_sub_topic_t  claim_index  
0  Police Car Partition                              0  
1  Consent for Searches                              1  


In [90]:
claims['claims'] = claims['claims'].str.replace(r'^\["', '', regex=True).str.replace(r'"\]$', '', regex=True).str.replace(r'"', '', regex=True).str.strip()
result_df['claim_index'] = result_df['claim_index'].astype(int)
claims = pd.merge(claims, result_df, how='left', on='claim_index')
print(claims.head(2))


   index  \
0   1101   
1   1101   

                                                                   claims  \
0  The police car has a partition to prevent the officer from reaching in   
1               The partition is to indicate lack of consent for searches   

          claim_topic  \
0   "police vehicles"   
1  'police procedure'   

                                                               thread_title_x  \
0  In all these videos, why does the person refuse to roll down their window?   
1  In all these videos, why does the person refuse to roll down their window?   

   claim_index  \
0            0   
1            1   

                                                               thread_title_y  \
0  In all these videos, why does the person refuse to roll down their window?   
1  In all these videos, why does the person refuse to roll down their window?   

          claim_topic_t claim_sub_topic_t  
0  Police Car Partition                    
1  Consent for Searches 

In [ ]:
claims.claim_topic_t.isna().sum()

np.int64(0)

In [ ]:
claims.claim_sub_topic_t.isna().sum()

np.int64(0)

In [ ]:
len(claims.claim_topic_t.unique())

35

In [ ]:
claims.claim_sub_topic_t.unique()

array(['', 'Consideration of Sex', 'Diabetes in Mexican-Americans',
       'Cervical Cancer in Vietnamese-American Women',
       'Prostate Cancer in Black Men',
       'Ethnic Minority Groups and Car Accidents',
       'Car Accident Rates in Ethnic Minority Groups',
       'Considering Race in Auto Insurance Policies',
       'Drawing Lines Between Males and Females',
       'Auto Insurance Rates for Men', 'Health Insurance Rates for Women',
       'Justification of Sex-Based Premiums',
       'Alternative Factors for Insurance Rates', 'Single-Payer System',
       "Women's Use of Healthcare Services", 'Causes of Auto Accidents',
       'Penalizing Men for Reckless Driving', 'Socialized Medicine',
       'Non-Malicious Discrimination', 'Private Healthcare',
       'Sexism and Non-Malicious Discrimination',
       'Actuary Tables and Accidents per Mile Driven',
       'General Trends vs. Individual Characteristics',
       'Unfair Charging of Young Men',
       'Individual Circumstance

In [ ]:
claims.claim_topic_t.unique()

array(['Police Car Partition', 'Consent for Searches',
       'Retaliation by Cops', 'Arrest Powers of Texas Cop',
       'K9 Unit Delaying Drivers', 'Ways to Retaliate Against Citizens',
       'Disgust with Exercising Rights', 'Abuse of Power Concerns',
       'Disregard for the Law', 'Suspicious Behavior by Officer',
       'Consequences of Refusing to Roll Down Window',
       'Guilt Until Proven Innocent',
       'Targeting People for Exercising Rights', 'Lack of Public Trust',
       'Even Enforcement of the Law',
       'Retaliation by Cops for Non-Compliance', 'Respect for Rights',
       'Open Carry Laws in Texas',
       'Preventing Police from Claiming Suspicious Behavior',
       'Rolling Down Window to Be Polite',
       'Rolling Down Window to Defy Authority', 'Radar Gun Malfunctions',
       'Impact of Radar Gun Malfunctions on Tickets',
       'Fairness in Speeding Fines',
       'Bias Against Defendants Questioning Police Equipment', '',
       'Police Power to Make Dr

In [ ]:
#now try to identify difference between claims per topic:



In [68]:
SYSTEM_diff: str = \
    """
# Instruction

Identify the sets of similar claims within a collection of claims. A claims belong to the same set if any of the following conditions are met:
1. They share a unifying broader theme
2. A claim specifies an aspects of another claim in the set
3. A claim elaborations or provides context to another claim in the set. 

Follow these steps:

1. You will receive a JSON object containing a collection of claims along with their respective indices enclosed in chevrons `<...>`.
2. Find the sets of similar claims. Ensure that each claim is assigned to only one set. A set can contain one or more claims.
3. Your response consists of a JSON object with the claim index and the corresponding similar set number.

Response format in JSON:

[
    {
        "claim_index": "1",
        "set_number": "1"	
    },
    {
        "claim_index": "2",
        "set_number": "1"
    }
]
	"""

In [91]:
#try mining both topics and subtopics:

# Initialize an empty list to store the results
chunked_result: typing.List[dict] = []

# Define the GROUPER
GROUPER = 'claim_topic_t'

# Group the data by 'thread_title'
groupeddata = claims.groupby(GROUPER)

# Process each group
for group, df in tqdm.tqdm(groupeddata):
    try:
        response = requests.post(
            'https://inf.cl.uni-trier.de/',
            json={
                'model': MODELsmall,
                'system': SYSTEM_diff,
                'prompt': f'Only respond with the JSON without any comments or introduction, "Claims":\n<{df.loc[:,['claim_index', 'claims']].to_json(orient='records')}>',
                'options': options_large
                }
            ).json()
        
        # Check if 'response' key exists in the JSON response
        if 'response' in response:
            for claim in json.loads(response['response']):
                chunked_result.append({
                    'claim_set': claim.get('set_number', 'unknown'),  # Use 'unknown' if 'topic' key is missing
                    'claim_index': claim.get('claim_index', 'unknown')  # Use 'unknown' if 'claim' key is missing
                })
        else:
            print(f"Key 'response' not found in the response: {response}")
    
    except json.JSONDecodeError:
        print("Invalid JSON response, skipping to next batch")
    except requests.RequestException as e:
        print(f"Request failed: {e}")

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(result_df.head(2))

100%|██████████| 35/35 [01:11<00:00,  2.04s/it]

  claim_set claim_index
0         1          57
1         1          69


In [92]:
result_df['claim_index'] = result_df['claim_index'].astype(int)
claims = pd.merge(claims, result_df.loc[:, ['claim_set','claim_index']], how='left', on='claim_index')
print(claims.head(2))

   index  \
0   1101   
1   1101   

                                                                   claims  \
0  The police car has a partition to prevent the officer from reaching in   
1               The partition is to indicate lack of consent for searches   

          claim_topic  \
0   "police vehicles"   
1  'police procedure'   

                                                               thread_title_x  \
0  In all these videos, why does the person refuse to roll down their window?   
1  In all these videos, why does the person refuse to roll down their window?   

   claim_index  \
0            0   
1            1   

                                                               thread_title_y  \
0  In all these videos, why does the person refuse to roll down their window?   
1  In all these videos, why does the person refuse to roll down their window?   

          claim_topic_t claim_sub_topic_t claim_set  
0  Police Car Partition                           1  
1  C

In [93]:
claims.claim_set.isna().sum()

np.int64(7)

In [95]:
claims.groupby(['claim_topic_t'])['claim_set'].count()

claim_topic_t
                                                        21
Abuse of Power Concerns                                  1
Arrest Powers of Texas Cop                               2
Assuming Guilt Inside a Car During Traffic Stop          1
Bias Against Defendants Questioning Police Equipment     1
Consent for Searches                                     1
Consequences of Refusing to Roll Down Window             1
Cop's Ability to See Inside the Car                      1
Discrimination                                           4
Disgust with Exercising Rights                           1
Disregard for the Law                                    1
Even Enforcement of the Law                              1
Fairness in Speeding Fines                               1
Guilt Until Proven Innocent                              1
Health Disparities                                       4
Healthcare System                                        5
Impact of Radar Gun Malfunctions on Ticket

In [96]:
claims.groupby(['claim_topic_t'])['claim_set'].nunique()

claim_topic_t
                                                        20
Abuse of Power Concerns                                  1
Arrest Powers of Texas Cop                               1
Assuming Guilt Inside a Car During Traffic Stop          1
Bias Against Defendants Questioning Police Equipment     1
Consent for Searches                                     1
Consequences of Refusing to Roll Down Window             1
Cop's Ability to See Inside the Car                      1
Discrimination                                           2
Disgust with Exercising Rights                           1
Disregard for the Law                                    1
Even Enforcement of the Law                              1
Fairness in Speeding Fines                               1
Guilt Until Proven Innocent                              1
Health Disparities                                       3
Healthcare System                                        3
Impact of Radar Gun Malfunctions on Ticket

In [81]:
result.groupby(['post_topic', 'claim_set']).size().reset_index(name='count')['count'].value_counts()    

count
1    81
2     2
3     1
Name: count, dtype: int64

In [ ]:
#81 unique 'sets' with only one claim in them... not very useful

In [ ]:
#maybe for now classify thread topics as political or not political and proceed with all comments in those threads, just assessing whether they offer new claims or not:

In [8]:
#create a dataset with all unique thread_titles
thread_titles = pd.DataFrame(reddit3000.thread_title.unique(), columns=['thread_title'])

In [9]:
thread_titles

,thread_title
0,DTX120: #87 - Nashville
1,"Male, 23 years old. Going for 100 sets!"
2,122cal black currant cheesecake!
3,Need help finding a Springfield!
4,Help with a possible trade?
...,...
257,Why is it that people have a mid-life crisis?
258,"What's your favorite book, show or movie line that you've adopted to use in your personal life"
259,What do you do when a friend bails on your plans?
260,What is your biggest heartache right now?


In [10]:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (thread_titles["thread_title"], MODELsmall, options=options_zero)
    )
    for label, path in CFG.prompt_classify_files.items() if label in ['political_post']
}

classifying political_post: 100%|██████████| 262/262 [03:56<00:00,  1.11it/s]


In [11]:
predictions

{'political_post': 0      non-political
 1      non-political
 2      non-political
 3      non-political
 4      non-political
            ...      
 257    non-political
 258    non-political
 259    non-political
 260    non-political
 261    non-political
 Name: political_post, Length: 262, dtype: object}

In [15]:
#join to the dataset:
for label, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    # Convert Series to DataFrame
    preds_df = preds.to_frame()
    # Rename the column in preds_df to include the label
    preds_df.columns = [f'Llama31_thread_{label}_8b']
    thread_titles = thread_titles.join(preds_df)

political_post
non-political    241
political         21
Name: count, dtype: int64
------------------------------------------


In [18]:
reddit3000 = reddit3000.join(thread_titles.set_index('thread_title'), on='thread_title')

In [ ]:
#now find out if the posts in these threads offer new information or not:
pol_threads = reddit3000.loc[reddit3000['Llama31_thread_political_post_8b'] == 'political',:]


In [20]:
pol_threads.head(3)

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI,Llama31_political_post_8b,Llama31_political_fill_8b,Llama31_political_fill_8b_score,Llama31_thread_political_post_8b
166,t3_3bxzpg,at corp in a decently sized ffa is ranging or meleeing better for getting drops?,2007scape,Corp FFA,none,question,True,NaN,k0bron,"[""ranging is better than meleeing for getting drops in a decently sized FFA at Corp"", ""meleeing ...","[0.48918917775154114, -0.22944757342338562, -0.5515434741973877, 0.5668707489967346, -0.24053360...",non-political,non-political,0,political
167,t1_csqkv45,Ranging,2007scape,Corp FFA,t3_3bxzpg,answer,None,1.0,Exaggeratethis,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713...",non-political,non-political,0,political
168,t1_csqkwye,if you have max gear and are 95+ combat stats probably melee,2007scape,Corp FFA,t3_3bxzpg,answer,None,1.0,None,"[""Having max gear and being 95+ in combat stats makes melee probable""]","[0.538628876209259, -0.29295164346694946, -0.32033681869506836, 0.5810739994049072, 0.0243992209...",non-political,non-political,0,political


In [35]:
SYSTEM_new: str = \
    """
# Instruction

Identify whether a social media posts adds new information (2), expands on information (1) or does not add new information (0) on the topic of a debate. New information is present (2) if the post introduces a new argument or perspective within the thread topic. 
A post expands on an existing argument or perspective (1) if it provides additional context or examples to an existing argument or perspective, or if it provides a sub-argument (parent-child relation). 
If the post does not relate to the thread topic or provides no new information, assign a value of 0. Follow these steps:


Follow these steps:

1. You will receive a target post in double chevrons <<...>> along with the thread topic in single chevrons <...>, and preceding posts enclosed in triple chevrons `<<<...>>>`.
2. Verify if the target post is on the thread topic.
3. Identify if new information is presented in the target post with respect to the preceding posts. 
4. If in doubt, assign a value of 0.

Respond with only the predicted class (0 or 1 or 2) of the request. Do not include any additional text or explanations.
Class:
	"""

In [65]:
SYSTEM_claim_new: str = \
    """
# Instruction

Identify whether a claim adds new information (2), expands on information (1) or does not add new information (0) with respect to preceding claims. New information is present (2) if the claim introduces a new argument or perspective. 
A claim expands on an existing argument or perspective (1) if it strongly related to it, or it provides additional context or examples to an argument or perspective in a preceding claim, or if it provides a sub-argument (parent-child relation). 
If the claim is very similar to a preceding claim or does not add any perspective or argument, assign a value of 0. Follow these steps:


Follow these steps:

1. You will receive a target claim in double chevrons <<...>> along with the preceding claims in single chevrons <...>.
2. Identify if new information is presented in the target claim with respect to the preceding posts. 
4. If in doubt, assign a value of 0.

Respond with only the predicted class (0 or 1 or 2) of the request. Do not include any additional text or explanations.
Class:
	"""

In [62]:
SYSTEM_claim_simscore: str = \
    """
# Instruction

Identify degree of similarity of a claim with respect to preceding claims on an intiger scale of 1 (completely dissimilar) to 10 (identical). A score of 5 indicates that the claims share context or topic, but are otherwise different. 
Follow these steps:

1. You will receive a target claim in double chevrons <<...>> along with a JSON containing the preceding claims and their claim_index enclosed in single chevrons <...>.
2. Determine the most similar claim to the target claim in the preceding claims. If only one preceding claim is provided pick this claim as the most similar claim.
3. Find the claim_index of this most similar claim. If you can't decide which claim is most similar, return an empty list [].
4. Identify the degree of similarity of the target claim with respect to that claim on a range of 1-10. 
5. If you can't give a similarity score return an empty list []. If no target claim is provided or only '[]' return an empty list [] for both the most_similar_claim_index and the similarity_score.
6. Only respond with the claim_index and similarity score.

Response format in JSON:

[
    {
        "most_similar_claim_index": "1",
        "similarity_score": "1"	
    }
]
	"""

In [66]:
#test low temperature claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = pol_threads[:10].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[[index,
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_new,
                                    'prompt': f'"Target post":<<{row["body"]}>>, "thread topic": <{row["thread_title"]}>, "preceding posts":\n<<<['']>>>',
                                    'options': options_zero
                                    }).json()['response']                       
                        ]],
                        columns=['index','new_information']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[[index,
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_new,
                                    'prompt': f'"Target post":<<{row["body"]}>>, "thread topic": <{row["thread_title"]}>, "preceding posts":\n<<<{df["body"][:threadset].to_list()}>>>',
                                    'options': options_zero
                                    }).json()['response']                       
                        ]],
                        columns=['index','new_information']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")

NameError: name 'pol_threads' is not defined

In [37]:
chunked_result

[   index new_information
 0    166               2,
    index new_information
 0    167               2,
    index new_information
 0    168               2,
    index new_information
 0    169               1,
    index new_information
 0    170               2,
    index new_information
 0    171               2,
    index new_information
 0    495               2,
    index new_information
 0    496               2,
    index new_information
 0    497               2,
    index new_information
 0    498               2]

In [34]:
pol_threads['body'][:10]

166                       at corp in a decently sized ffa is ranging or meleeing better for getting drops?
167                                                                                                Ranging
168                                           if you have max gear and are 95+ combat stats probably melee
169    Depending on the size, range *can* be better than melee. In a very large group all it takes is 2...
170                                                                                        whos right?\n\n
171                                                         Both can win. Ranging will probably cost more.
495                                                                                                       
496    My favorite Chip Kelly gif: http://i.kinja-img.com/gawker-media/image/upload/s--WTcuZUsN--/c_fit...
497                                                   I want my flair to be him with his tongue on the mic
498                                  

In [ ]:
#doesn't appear to discriminate well between new information and not new information, considering 'whos right?' and 'None' as new information	...

In [41]:
claims.head()

,index,claims,claim_topic,thread_title
0,1101,"[""The police car has a partition to prevent the officer from reaching in","""police vehicles""","In all these videos, why does the person refuse to roll down their window?"
1,1101,"""The partition is to indicate lack of consent for searches""]",'police procedure',"In all these videos, why does the person refuse to roll down their window?"
2,1102,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","""Cop behavior and accountability""","In all these videos, why does the person refuse to roll down their window?"
3,1102,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""Police powers""","In all these videos, why does the person refuse to roll down their window?"
4,1102,"""Some cops suggest delaying drivers until a K9 unit arrives""]","""Law Enforcement Tactics""","In all these videos, why does the person refuse to roll down their window?"


In [167]:
#maybe try to mine claims first and then assess new information per claim:
#test low temperature claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = claims[:10].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[[index,
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_new,
                                    'prompt': f'"Target claim":<<{row["claims"]}>>, "preceding claims":\n<['']>',
                                    'options': options_zero
                                    }).json()['response']                       
                        ]],
                        columns=['index','new_information']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[[index,
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_new,
                                    'prompt': f'"Target claim":<<{row["claims"]}>>, "preceding claims":\n<{df["claims"][:threadset].to_list()}>',
                                    'options': options_zero
                                    }).json()['response']                       
                        ]],
                        columns=['index','new_information']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")


100%|██████████| 1/1 [00:13<00:00, 13.15s/it]


In [62]:
chunked_result

[   index new_information
 0      0              99,
    index new_information
 0      1              99,
    index new_information
 0      2              99,
    index new_information
 0      3               3,
    index new_information
 0      4               3,
    index new_information
 0      5               4,
    index new_information
 0      6               3,
    index new_information
 0      7               4,
    index new_information
 0      8               4,
    index new_information
 0      9               4]

In [50]:
#same result

In [5]:
#load claims:
claims: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/reddit3000_pol_posts_claims.parquet')
claims.loc[:, 'claim_index'] = claims.index

In [80]:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = claims[1:10].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            print(threadset)
            print(f'"Target claim":<<{row["claims"]}>>, "preceding claims":\n<{df.loc[:,['claim_index', 'claims']][:threadset].to_json(orient='records')}>')

100%|██████████| 1/1 [00:00<00:00, 166.56it/s]

1
"Target claim":<<["Cops are discussing retaliating against drivers who exercise their rights during traffic stops>>, "preceding claims":
<[{"claim_index":1,"claims":" \"The partition is to indicate lack of consent for searches\"]"}]>
2
"Target claim":<< "A Texas cop claims he can arrest for any offense other than speeding or open carry>>, "preceding claims":
<[{"claim_index":1,"claims":" \"The partition is to indicate lack of consent for searches\"]"},{"claim_index":2,"claims":"[\"Cops are discussing retaliating against drivers who exercise their rights during traffic stops"}]>
3
"Target claim":<< "Some cops suggest delaying drivers until a K9 unit arrives"]>>, "preceding claims":
<[{"claim_index":1,"claims":" \"The partition is to indicate lack of consent for searches\"]"},{"claim_index":2,"claims":"[\"Cops are discussing retaliating against drivers who exercise their rights during traffic stops"},{"claim_index":3,"claims":" \"A Texas cop claims he can arrest for any offense other t

In [ ]:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = claims[1:10].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(pd.DataFrame(
                    data=[[requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_claim_simscore,
                                    'prompt': f'"Target claim":<<{row["claims"]}>>, "preceding claims and indices":\n<{df.loc[:,['claim_index', 'claims']][:threadset].to_json(orient='records')}>',
                                    'options': options_zero
                                    }).json()['response']]],
                    columns=['similariy']
                ))
                
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")

similarity = pd.concat(chunked_result, ignore_index=True)
print(similarity)

100%|██████████| 1/1 [00:25<00:00, 25.24s/it]


In [ ]:
print(similarity)

                                                                                             similariy
0  [\n    {\n        "most_similar_claim_index": "1",\n        "similarity_score": "8"\n    }\n]\n ...
1  [\n    {\n        "most_similar_claim_index": "2",\n        "similarity_score": "8"\n    }\n]\n ...
2  [\n    {\n        "most_similar_claim_index": "2",\n        "similarity_score": "8"\n    }\n]\n ...
3  [\n    {\n        "most_similar_claim_index": "2",\n        "similarity_score": "8"\n    }\n]\n ...
4  [\n    {\n        "most_similar_claim_index": "2",\n        "similarity_score": "8"\n    }\n]\n ...
5  [\n    {\n        "most_similar_claim_index": "2",\n        "similarity_score": "8"\n    }\n]\n ...
6  [\n    {\n        "most_similar_claim_index": "5",\n        "similarity_score": "8"\n    }\n]\n ...
7  [\n    {\n        "most_similar_claim_index": "5",\n        "similarity_score": "9"\n    }\n]\n ...


In [107]:
similarity.similariy.to_list()

['[\n    {\n        "most_similar_claim_index": "1",\n        "similarity_score": "8"\n    }\n]\n \nExplanation: The target claim mentions exercising rights during traffic stops, which is similar to the preceding claim about indicating lack of consent for searches. I\'ve given a similarity score of 8 as it\'s not identical but has a clear connection between the two concepts.',
 '[\n    {\n        "most_similar_claim_index": "2",\n        "similarity_score": "8"\n    }\n]\n \nThe target claim is most similar to the second preceding claim, as both involve interactions between law enforcement and individuals exercising their rights. The similarity score of 8 reflects a strong connection between the two claims, but not an exact match (which would be a score of 10).',
 '[\n    {\n        "most_similar_claim_index": "2",\n        "similarity_score": "8"\n    }\n]\n \nThe target claim is most similar to the second preceding claim, which discusses cops discussing retaliation against drivers. T

In [150]:
claims.head(3)

,index,claims_x,claim_topic,thread_title,claim_index,most_similar_claim_index,similarity_score,most_similar_claim
0,1101,"[""The police car has a partition to prevent the officer from reaching in","""police vehicles""","In all these videos, why does the person refuse to roll down their window?",0,1,10,"""The partition is to indicate lack of consent for searches""]"
1,1101,"""The partition is to indicate lack of consent for searches""]",'police procedure',"In all these videos, why does the person refuse to roll down their window?",1,99,99,NaN
2,1102,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","""Cop behavior and accountability""","In all these videos, why does the person refuse to roll down their window?",2,0,1,"[""The police car has a partition to prevent the officer from reaching in"


In [15]:
claims.loc[claims.claims == '[]', 'claim_index']

27    27
33    33
52    52
53    53
54    54
59    59
61    61
Name: claim_index, dtype: int64

In [63]:
#try asking for a similarity score instead:

chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = claims.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.claim_index.shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append({
                "claim_index": row['claim_index'],
                "most_similar_claim_index": '',
                'similarity_score': ''  
                    })
                
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            if row['claims'] == '[]':
                chunked_result.append({
                    "claim_index": row['claim_index'],
                    "most_similar_claim_index": '',
                    'similarity_score': ''  
                    })
            else:
                try: 
                    response = requests.post(
                                    'https://inf.cl.uni-trier.de/',
                                    json={
                                        'model': MODELsmall,
                                        'system': SYSTEM_claim_simscore,
                                        'prompt': f'Reason about your respons, but respond with nothing else than the JSON. "Target claim":<<{row["claims"]}>>, "preceding claims and indices":\n<{df.loc[:,['claim_index', 'claims']][:threadset].to_json(orient='records')}>',
                                        'options': options_zero
                                        }).json()
                    # Check if 'response' key exists in the JSON response
                    if 'response' in response:
                        for claim in json.loads(response['response']):
                            chunked_result.append({
                                "claim_index": row['claim_index'],
                                "most_similar_claim_index": claim.get('most_similar_claim_index', ''),
                                'similarity_score': claim.get('similarity_score', '')  
                        })
                    else:
                        print(f"Key 'response' not found in the response: {response}")                      
    
                except json.JSONDecodeError:
                    print("invalid json response, skipping to next batch")

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(result_df.head(4))

100%|██████████| 2/2 [01:00<00:00, 30.48s/it]

   claim_index most_similar_claim_index similarity_score
0            0                                          
1            1                                          
2            2                                          
3            3                        1                5


In [64]:
result_df.head(10)

,claim_index,most_similar_claim_index,similarity_score
0,0,,
1,1,,
2,2,,
3,3,1,5
4,4,0,5
5,5,2,8
6,6,2,5
7,7,2,8
8,8,1,8
9,9,5,8


In [51]:
result_df.similarity_score.value_counts()

similarity_score
10    49
8     20
      12
9      7
2      1
Name: count, dtype: int64

In [52]:
result_df.loc[result_df.similarity_score=='2', :]

,claim_index,most_similar_claim_index,similarity_score
3,3,1,2


In [53]:
claims.columns

Index(['index', 'claims', 'claim_topic', 'thread_title', 'claim_index',
       'most_similar_claim_index', 'similarity_score', 'most_similar_claim'],
      dtype='object')

In [54]:
claims.drop(columns=[ 'most_similar_claim_index', 'similarity_score', 'most_similar_claim'], inplace=True)

In [55]:
result_df['claim_index'] = result_df['claim_index'].astype(int)
claims = pd.merge(claims, result_df, how='left', on='claim_index')
print(claims.head(2))

   index  \
0   1101   
1   1101   

                                                                     claims  \
0  ["The police car has a partition to prevent the officer from reaching in   
1              "The partition is to indicate lack of consent for searches"]   

          claim_topic  \
0   "police vehicles"   
1  'police procedure'   

                                                                 thread_title  \
0  In all these videos, why does the person refuse to roll down their window?   
1  In all these videos, why does the person refuse to roll down their window?   

   claim_index most_similar_claim_index similarity_score  
0            0                                            
1            1                                            


In [56]:
claims.head(10)

,index,claims,claim_topic,thread_title,claim_index,most_similar_claim_index,similarity_score
0,1101,"[""The police car has a partition to prevent the officer from reaching in","""police vehicles""","In all these videos, why does the person refuse to roll down their window?",0,,
1,1101,"""The partition is to indicate lack of consent for searches""]",'police procedure',"In all these videos, why does the person refuse to roll down their window?",1,,
2,1102,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","""Cop behavior and accountability""","In all these videos, why does the person refuse to roll down their window?",2,,
3,1102,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""Police powers""","In all these videos, why does the person refuse to roll down their window?",3,1,2
4,1102,"""Some cops suggest delaying drivers until a K9 unit arrives""]","""Law Enforcement Tactics""","In all these videos, why does the person refuse to roll down their window?",4,0,8
5,1103,"[""Cops are discussing ways to retaliate against citizens",['police relations'],"In all these videos, why does the person refuse to roll down their window?",5,2,8
6,1103,"""exercising their rights is disgusting",[],"In all these videos, why does the person refuse to roll down their window?",6,2,8
7,1103,"""retaliation raises serious concerns about abuse of power","""Abuse of Power""","In all these videos, why does the person refuse to roll down their window?",7,2,8
8,1103,"""there's disregard for the law""]",'disregard for the law',"In all these videos, why does the person refuse to roll down their window?",8,2,8
9,1104,"[""Refusing to roll down your window more than two inches can be seen as suspicious behavior by an officer","""suspicious behavior""","In all these videos, why does the person refuse to roll down their window?",9,5,8


In [57]:
claims.loc[:, 'most_similar_claim_index'] = claims.most_similar_claim_index.replace("", 9999).astype(int)
#join the claim of the most similar claim to the dataset:
claims = pd.merge(claims, claims.loc[:,['claim_index', 'claims']], how='left', left_on='most_similar_claim_index', right_on='claim_index')
# Rename the column 'most_similar_claim_index' to 'most similar claim'
claims = claims.rename(columns={'claims_y': 'most_similar_claim', 'claim_index_x': 'claim_index', 'claims_x': 'claims'})

# Drop the 'claim_index_y' column if it exists
claims = claims.drop(columns=['claim_index_y'], errors='ignore')

In [58]:
claims.loc[:, 'similarity_score'].value_counts()

similarity_score
10    49
8     20
      12
9      7
2      1
Name: count, dtype: int64

In [59]:
#many duplicate (10) or very similar claims (8-9), only one very dissimilar claim (2)


In [60]:
claims.head(5)

,index,claims,claim_topic,thread_title,claim_index,most_similar_claim_index,similarity_score,most_similar_claim
0,1101,"[""The police car has a partition to prevent the officer from reaching in","""police vehicles""","In all these videos, why does the person refuse to roll down their window?",0,9999,,NaN
1,1101,"""The partition is to indicate lack of consent for searches""]",'police procedure',"In all these videos, why does the person refuse to roll down their window?",1,9999,,NaN
2,1102,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","""Cop behavior and accountability""","In all these videos, why does the person refuse to roll down their window?",2,9999,,NaN
3,1102,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""Police powers""","In all these videos, why does the person refuse to roll down their window?",3,1,2,"""The partition is to indicate lack of consent for searches""]"
4,1102,"""Some cops suggest delaying drivers until a K9 unit arrives""]","""Law Enforcement Tactics""","In all these videos, why does the person refuse to roll down their window?",4,0,8,"[""The police car has a partition to prevent the officer from reaching in"


In [61]:
claims.loc[:, ['claims', 'most_similar_claim', 'similarity_score']]	


,claims,most_similar_claim,similarity_score
0,"[""The police car has a partition to prevent the officer from reaching in",NaN,
1,"""The partition is to indicate lack of consent for searches""]",NaN,
2,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops",NaN,
3,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""The partition is to indicate lack of consent for searches""]",2
4,"""Some cops suggest delaying drivers until a K9 unit arrives""]","[""The police car has a partition to prevent the officer from reaching in",8
5,"[""Cops are discussing ways to retaliate against citizens","[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops",8
6,"""exercising their rights is disgusting","[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops",8
7,"""retaliation raises serious concerns about abuse of power","[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops",8
8,"""there's disregard for the law""]","[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops",8
9,"[""Refusing to roll down your window more than two inches can be seen as suspicious behavior by an officer","[""Cops are discussing ways to retaliate against citizens",8


In [23]:
print(claims.iloc[3].claims)
print(claims.iloc[1].claims)


 "A Texas cop claims he can arrest for any offense other than speeding or open carry
 "The partition is to indicate lack of consent for searches"]


In [ ]:
#I don't see any similarity between these two claims, so the similarity score is not always reliable

In [28]:
#let's look at the one dissimilar claim:
claims.loc[claims.similarity_score=='1', :].loc[:, ['claims', 'most_similar_claim', 'similarity_score']]

,claims,most_similar_claim,similarity_score
2,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","[""The police car has a partition to prevent the officer from reaching in",1


In [34]:
claims.iloc[2:4]

,index,claims,claim_topic,thread_title,claim_index,most_similar_claim_index,similarity_score,most_similar_claim
2,1102,"[""Cops are discussing retaliating against drivers who exercise their rights during traffic stops","""Cop behavior and accountability""","In all these videos, why does the person refuse to roll down their window?",2,0,1,"[""The police car has a partition to prevent the officer from reaching in"
3,1102,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""Police powers""","In all these videos, why does the person refuse to roll down their window?",3,1,10,"""The partition is to indicate lack of consent for searches""]"


In [42]:
#difficult to understand this as well, let's ask the model for feedback on these two claims:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = claims.iloc[:4].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(pd.DataFrame(
                    data=[[requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODELsmall,
                                    'system': SYSTEM_claim_simscore,
                                    'prompt': f'Please explain your reasoning. "Target claim":<<{row["claims"]}>>, "preceding claims and indices":\n<{df.loc[:,['claim_index', 'claims']][:threadset].to_json(orient='records')}>',
                                    'options': options_zero
                                    }).json()['response']]],
                    columns=['similariy']
                ))
                
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")

similarity = pd.concat(chunked_result, ignore_index=True)
print(similarity)

100%|██████████| 1/1 [00:10<00:00, 10.66s/it]

                                                                                             similariy
0  Since there are no preceding claims provided, I will return an empty list [] as per the instruct...
1  To determine the degree of similarity between the target claim and the preceding claims, I will ...
2  To determine the degree of similarity between the target claim and the preceding claims, I will ...


In [44]:
# Set display options to print the full DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# Print the full DataFrame
print(similarity)

In [ ]:
#apperently if you ask the model to reason, it corrects itself...

In [24]:
claims.loc[claims.similarity_score=='10', :].loc[:, ['claims', 'most_similar_claim', 'similarity_score']]

,claims,most_similar_claim,similarity_score
3,"""A Texas cop claims he can arrest for any offense other than speeding or open carry","""The partition is to indicate lack of consent for searches""]",10
35,"""Mexican-Americans are nearly twice as likely to have diabetes as whites","""Mexican-Americans are nearly twice as likely to have diabetes as whites",10
36,"""Vietnamese-American women have nearly five times the rate of cervical cancer as white women","""Vietnamese-American women have nearly five times the rate of cervical cancer as white women",10
37,"""Black men are twice as likely to have prostate cancer as white men","""Black men are twice as likely to have prostate cancer as white men",10
38,"""Certain ethnic minority groups in the U.K. are more likely to get injured than other ethnic mi...","""Certain ethnic minority groups in the U.K. are more likely to get injured than other ethnic mi...",10
39,"""This means they get into more car accidents","""This means they get into more car accidents",10
40,"""We should take race into consideration when forming auto insurance policies","""We should take race into consideration when forming auto insurance policies",10
41,"""It is acceptable to draw lines between males and females but not for everybody else""]","""It is acceptable to draw lines between males and females but not for everybody else""]",10
42,"[""Auto insurance rates are higher for men because they drive more miles than women.","[""Auto insurance rates are higher for men because they drive more miles than women.",10
43,"""Health insurance rates are higher for women because they use more healthcare services, particu...","""Health insurance rates are higher for women because they use more healthcare services, particu...",10


In [ ]:
#many correctly identified duplicate claims

In [25]:
#save data:
claims.to_parquet(f'{CFG.report_dir}/reddit3000_pol_posts_claims.parquet')

In [45]:
#maybe try the large LLAMA model to see if it can better discriminate between claims:
MODELlarge: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'


In [46]:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = claims.groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.claim_index.shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append({
                "claim_index": row['claim_index'],
                "most_similar_claim_index": '',
                'similarity_score': ''  
                    })
                
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            if row['claims'] == '[]':
                chunked_result.append({
                    "claim_index": row['claim_index'],
                    "most_similar_claim_index": '',
                    'similarity_score': ''  
                    })
            else:
                try: 
                    response = requests.post(
                                    'https://inf.cl.uni-trier.de/',
                                    json={
                                        'model': MODELlarge,
                                        'system': SYSTEM_claim_simscore,
                                        'prompt': f'Respond with nothing else than the JSON. "Target claim":<<{row["claims"]}>>, "preceding claims and indices":\n<{df.loc[:,['claim_index', 'claims']][:threadset].to_json(orient='records')}>',
                                        'options': options_zero
                                        }).json()
                    # Check if 'response' key exists in the JSON response
                    if 'response' in response:
                        for claim in json.loads(response['response']):
                            chunked_result.append({
                                "claim_index": row['claim_index'],
                                "most_similar_claim_index": claim.get('most_similar_claim_index', ''),
                                'similarity_score': claim.get('similarity_score', '')  
                        })
                    else:
                        print(f"Key 'response' not found in the response: {response}")                      
    
                except json.JSONDecodeError:
                    print("invalid json response, skipping to next batch")

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(chunked_result)

# Display the DataFrame to verify the changes
print(result_df.head(4))

100%|██████████| 2/2 [04:47<00:00, 143.66s/it]

   claim_index most_similar_claim_index similarity_score
0            0                                          
1            1                                          
2            2                                          
3            3                                          


In [47]:
result_df.head(10)

,claim_index,most_similar_claim_index,similarity_score
0,0,,
1,1,,
2,2,,
3,3,,
4,4,2,8
5,5,2,9
6,6,2,8
7,7,2,8
8,8,2,8
9,9,5,8


In [48]:
result_df.similarity_score.value_counts()

similarity_score
10    46
      19
8     18
9      6
Name: count, dtype: int64

In [ ]:
#larger model mainly assigns missing, to previously false values...

In [ ]:
result_df['claim_index'] = result_df['claim_index'].astype(int)
claims = pd.merge(claims, result_df, how='left', on='claim_index', suffixes=('', '_large'))
print(claims.head(2))

In [ ]:
claims.loc[:, 'most_similar_claim_index_large'] = claims.most_similar_claim_index_large.replace("", 9999).astype(int)
#join the claim of the most similar claim to the dataset:
claims = pd.merge(claims, claims.loc[:,['claim_index', 'claims']], how='left', left_on='most_similar_claim_index_large', right_on='claim_index', suffixes=('', '_largemerge'))
# Rename the column 'most_similar_claim_index' to 'most similar claim'
#claims = claims.rename(columns={'claims_y': 'most_similar_claim', 'claim_index_x': 'claim_index', 'claims_x': 'claims'})

# Drop the 'claim_index_y' column if it exists
claims = claims.drop(columns=['claim_index_largemerge'], errors='ignore')

In [ ]:
#Is political opinion, topic, group topics
#Get extended comments from set, which are different, which are similar, which extensions/additions